# Import libraries

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.


In [2]:
df = pd.read_csv("../datasets/final_data/FIFA Data 1990-2018.csv")
df_teams = pd.read_csv("../datasets/final_data/world_cup_comparisons.csv")
fifa_ranking = pd.read_csv("../datasets/final_data/fifa_ranking.csv")
continent = pd.read_csv("../datasets/final_data/countries and continents1.csv",encoding ='utf-8',na_filter = False)


df=df.replace({"South Korea":"Korea Republic"})
df = df.replace({"Iran":"IR Iran"})
df = df.replace({"Soviet Union":"Russia"})
df = df.replace({"Czechoslovakia":"Czech Republic"})


df_teams=df_teams.replace({"South Korea":"Korea Republic"})
df_teams = df_teams.replace({"Iran":"IR Iran"})

fifa_ranking=fifa_ranking.replace({"South Korea":"Korea Republic"})
fifa_ranking = fifa_ranking.replace({"Iran":"IR Iran"})


In [3]:
df.head()

,Team1,Team2,Rounds,Year,Score1,Score2,Ranking T1,Score T1,Ranking T2,Score T2,Winner
0,Russia,Saudi Arabia,Group Phase,2018,5,0,70.0,457.0,67.0,465.0,1
1,Egypt,Uruguay,Group Phase,2018,0,1,45.0,649.0,14.0,1018.0,2
2,Morocco,IR Iran,Group Phase,2018,0,1,41.0,686.0,37.0,708.0,2
3,Portugal,Spain,Group Phase,2018,3,3,4.0,1274.0,10.0,1126.0,0
4,France,Australia,Group Phase,2018,2,1,7.0,1198.0,36.0,718.0,1


In [4]:
df = df.drop(['Winner'], axis =1)

# Duplicate

In [5]:

# we should change all respective column for duplicate
for index,row in df.iterrows():
    df.set_value(index+488,["Team1","Team2","Rounds","Year","Score1","Score2",'Ranking T1','Score T1','Ranking T2','Score T2'],[row["Team2"],row["Team1"],row["Rounds"],row["Year"],row["Score2"],row["Score1"],row['Ranking T2'],row['Score T2'],row['Ranking T1'],row['Score T1']])
df.shape


/home/milad/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  after removing the cwd from sys.path.


(976, 10)

In [6]:
df.iloc[0]

Team1               Russia
Team2         Saudi Arabia
Rounds         Group Phase
Year                  2018
Score1                   5
Score2                   0
Ranking T1              70
Score T1               457
Ranking T2              67
Score T2               465
Name: 0, dtype: object

In [7]:
df.iloc[488]

Team1         Saudi Arabia
Team2               Russia
Rounds         Group Phase
Year                  2018
Score1                   0
Score2                   5
Ranking T1              67
Score T1               465
Ranking T2              70
Score T2               457
Name: 488, dtype: object

# Team Feature



In [8]:
features=[]
for key in df_teams.keys():
    features.append(str(key))

features

del features[0:4]

features


['xg_z',
 'crosses_z',
 'boxtouches_z',
 'passes_z',
 'progpasses_z',
 'takeons_z',
 'progruns_z',
 'tackles_z',
 'interceptions_z',
 'clearances_z',
 'blocks_z',
 'aerials_z',
 'fouls_z',
 'fouled_z',
 'nsxg_z']

In [9]:
df_teams_avg = df_teams.groupby(['Year','team'])['goals_z'].sum()/df_teams.groupby(['Year','team'])['goals_z'].size()
df_teams_avg = df_teams_avg.to_frame()

df_teams_avg['Year'] = df_teams_avg.index.get_level_values(0).tolist()
df_teams_avg['team'] = df_teams_avg.index.get_level_values(1).tolist()


df_teams_avg.head()


goals_z  Year       team
Year team                                
1966 Argentina  0.049167  1966  Argentina
     Brazil    -0.038000  1966     Brazil
     Bulgaria  -0.280000  1966   Bulgaria
     Chile     -0.139286  1966      Chile
     England    0.221333  1966    England

In [10]:
def avg_team(feature,df_teams_avg):
    a=df_teams.groupby(['Year','team'])[feature].sum()/df_teams.groupby(['Year','team'])[feature].size()
    b=a.to_frame()
    df_teams_avg[feature]= b[feature]
    

for feature in features:
    avg_team(feature,df_teams_avg)

df_teams_avg.head()

goals_z  Year       team      xg_z  crosses_z  boxtouches_z  \
Year team                                                                      
1966 Argentina  0.049167  1966  Argentina  0.021667  -0.440833     -0.138333   
     Brazil    -0.038000  1966     Brazil -0.185500  -0.249500     -0.097000   
     Bulgaria  -0.280000  1966   Bulgaria -0.206875  -0.335000     -0.199375   
     Chile     -0.139286  1966      Chile  0.160714  -0.050000      0.007857   
     England    0.221333  1966    England  0.016000   0.424000      0.221333   

                passes_z  progpasses_z  takeons_z  progruns_z  tackles_z  \
Year team                                                                  
1966 Argentina  0.836667      0.265000   0.621667    0.572500   0.090000   
     Brazil    -0.145500     -0.179000   0.139500   -0.237500   0.017000   
     Bulgaria  -0.128125     -0.018125   0.344375   -0.025000  -0.235000   
     Chile      0.515714      0.370714   0.130714    0.074286   0.407143   
     England    0.019333     -0.035333   0.048667    0.274667  -0.225333   

                interceptions_z  clearances_z  blocks_z  aerials_z   fouls_z  \
Year team                                                                      
1966 Argentina         0.501667      0.219167 -0.064167  -0.134167  0.522500   
     Brazil            0.039000     -0.275500 -0.229000  -0.255500 -0.141500   
     Bulgaria         -0.583125     -0.096250 -0.108125  -0.302500 -0.072500   
     Chile             0.215714     -0.045714  0.260000  -0.259286  0.090000   
     England          -0.218667      0.046667  0.160667   0.450000  0.189333   

                fouled_z    nsxg_z  
Year team                           
1966 Argentina  0.565833 -0.110833  
     Brazil    -0.240500 -0.016500  
     Bulgaria   0.029375 -0.013750  
     Chile      0.104286  0.019286  
     England   -0.310667  0.136000

In [11]:
new_df = pd.merge(df, df_teams_avg,  how='left', left_on=['Team1','Year'], right_on = ['team','Year'])
new_df1 = pd.merge(new_df,df_teams_avg ,  how='left', left_on=['Team2','Year'], right_on = ['team','Year'])


new_df1.head()

,Team1,Team2,Rounds,Year,Score1,Score2,Ranking T1,Score T1,Ranking T2,Score T2,...,takeons_z_y,progruns_z_y,tackles_z_y,interceptions_z_y,clearances_z_y,blocks_z_y,aerials_z_y,fouls_z_y,fouled_z_y,nsxg_z_y
0,Russia,Saudi Arabia,Group Phase,2018,5.0,0.0,70.0,457.0,67.0,465.0,...,-0.136316,0.073684,0.512632,0.126316,-0.246316,0.209474,-0.466316,-0.387368,0.051579,-0.294211
1,Egypt,Uruguay,Group Phase,2018,0.0,1.0,45.0,649.0,14.0,1018.0,...,-0.080625,0.025000,-0.043750,0.891875,0.015000,-0.056250,-0.066875,-0.130000,0.116250,0.041875
2,Morocco,IR Iran,Group Phase,2018,0.0,1.0,41.0,686.0,37.0,708.0,...,-0.106250,-0.435625,-0.701250,-0.608750,0.289375,-0.327500,-0.082500,0.200000,0.236250,-0.388125
3,Portugal,Spain,Group Phase,2018,3.0,3.0,4.0,1274.0,10.0,1126.0,...,0.428235,0.800000,0.513529,0.456471,-0.374706,-0.008824,-0.140000,-0.400000,0.145882,0.644706
4,France,Australia,Group Phase,2018,2.0,1.0,7.0,1198.0,36.0,718.0,...,0.097333,-0.029333,0.031333,-0.022667,0.221333,-0.341333,0.240667,0.096000,-0.071333,0.004000


In [12]:
df = new_df1
df.isnull().sum()

Team1                 0
Team2                 0
Rounds                0
Year                  0
Score1                0
Score2                0
Ranking T1            0
Score T1              0
Ranking T2            0
Score T2              0
goals_z_x            50
team_x               50
xg_z_x               50
crosses_z_x          50
boxtouches_z_x       50
passes_z_x           50
progpasses_z_x       50
takeons_z_x          50
progruns_z_x         50
tackles_z_x          50
interceptions_z_x    50
clearances_z_x       50
blocks_z_x           50
aerials_z_x          50
fouls_z_x            50
fouled_z_x           50
nsxg_z_x             50
goals_z_y            50
team_y               50
xg_z_y               50
crosses_z_y          50
boxtouches_z_y       50
passes_z_y           50
progpasses_z_y       50
takeons_z_y          50
progruns_z_y         50
tackles_z_y          50
interceptions_z_y    50
clearances_z_y       50
blocks_z_y           50
aerials_z_y          50
fouls_z_y       

In [13]:
df = df.dropna(axis=0, subset=['goals_z_x','goals_z_y'])
df.isnull().sum()

Team1                0
Team2                0
Rounds               0
Year                 0
Score1               0
Score2               0
Ranking T1           0
Score T1             0
Ranking T2           0
Score T2             0
goals_z_x            0
team_x               0
xg_z_x               0
crosses_z_x          0
boxtouches_z_x       0
passes_z_x           0
progpasses_z_x       0
takeons_z_x          0
progruns_z_x         0
tackles_z_x          0
interceptions_z_x    0
clearances_z_x       0
blocks_z_x           0
aerials_z_x          0
fouls_z_x            0
fouled_z_x           0
nsxg_z_x             0
goals_z_y            0
team_y               0
xg_z_y               0
crosses_z_y          0
boxtouches_z_y       0
passes_z_y           0
progpasses_z_y       0
takeons_z_y          0
progruns_z_y         0
tackles_z_y          0
interceptions_z_y    0
clearances_z_y       0
blocks_z_y           0
aerials_z_y          0
fouls_z_y            0
fouled_z_y           0
nsxg_z_y   

In [26]:
df.head(150)

,Team1,Team2,Rounds,Year,Score1,Score2,Ranking T1,Score T1,Ranking T2,Score T2,...,takeons_z_y,progruns_z_y,tackles_z_y,interceptions_z_y,clearances_z_y,blocks_z_y,aerials_z_y,fouls_z_y,fouled_z_y,nsxg_z_y
0,Russia,Saudi Arabia,Group Phase,2018,5.0,0.0,70.0,457.0,67.0,465.0,...,-0.136316,0.073684,0.512632,0.126316,-0.246316,0.209474,-0.466316,-3.873684e-01,0.051579,-0.294211
1,Egypt,Uruguay,Group Phase,2018,0.0,1.0,45.0,649.0,14.0,1018.0,...,-0.080625,0.025000,-0.043750,0.891875,0.015000,-0.056250,-0.066875,-1.300000e-01,0.116250,0.041875
2,Morocco,IR Iran,Group Phase,2018,0.0,1.0,41.0,686.0,37.0,708.0,...,-0.106250,-0.435625,-0.701250,-0.608750,0.289375,-0.327500,-0.082500,2.000000e-01,0.236250,-0.388125
3,Portugal,Spain,Group Phase,2018,3.0,3.0,4.0,1274.0,10.0,1126.0,...,0.428235,0.800000,0.513529,0.456471,-0.374706,-0.008824,-0.140000,-4.000000e-01,0.145882,0.644706
4,France,Australia,Group Phase,2018,2.0,1.0,7.0,1198.0,36.0,718.0,...,0.097333,-0.029333,0.031333,-0.022667,0.221333,-0.341333,0.240667,9.600000e-02,-0.071333,0.004000
5,Argentina,Iceland,Group Phase,2018,1.0,1.0,5.0,1241.0,22.0,908.0,...,-0.094667,-0.248667,-0.629333,-0.490000,0.377333,-0.156000,0.220667,3.700743e-18,-0.237333,0.039333
6,Peru,Denmark,Group Phase,2018,0.0,1.0,11.0,1125.0,12.0,1051.0,...,-0.135882,-0.294706,-0.269412,-0.273529,0.364118,0.077647,0.441176,2.352941e-03,-0.419412,-0.069412
7,Croatia,Nigeria,Group Phase,2018,2.0,0.0,20.0,945.0,48.0,618.0,...,0.509333,-0.188667,-0.042000,0.118667,0.220000,-0.186000,0.296667,2.240000e-01,0.286000,-0.092000
8,Costa Rica,Serbia,Group Phase,2018,0.0,1.0,23.0,884.0,34.0,751.0,...,0.183571,0.263571,0.073571,-0.379286,0.156429,0.186429,0.506429,4.228571e-01,0.195000,0.485714
9,Germany,Mexico,Group Phase,2018,0.0,1.0,1.0,1558.0,15.0,989.0,...,0.594286,0.821429,0.475714,0.396429,0.047857,0.884286,-0.028571,8.000000e-02,0.282857,0.245000


# Add lenth of name for each country

In [27]:
df.to_csv("./temp.csv")

In [29]:
df = pd.read_csv("./temp.csv")

In [31]:
len_Team1 =[]
len_Team2 =[]

for i in range (len(df['Team1'])):
    len_Team1.append(len(df['Team1'][i]))
    len_Team2.append(len(df['Team2'][i]))
    
df['len_Team1'] =len_Team1
df['len_Team2'] = len_Team2

df.head()
    

,Unnamed: 0,Team1,Team2,Rounds,Year,Score1,Score2,Ranking T1,Score T1,Ranking T2,...,tackles_z_y,interceptions_z_y,clearances_z_y,blocks_z_y,aerials_z_y,fouls_z_y,fouled_z_y,nsxg_z_y,len_Team1,len_Team2
0,0,Russia,Saudi Arabia,Group Phase,2018.0,5.0,0.0,70.0,457.0,67.0,...,0.512632,0.126316,-0.246316,0.209474,-0.466316,-0.387368,0.051579,-0.294211,6,12
1,1,Egypt,Uruguay,Group Phase,2018.0,0.0,1.0,45.0,649.0,14.0,...,-0.043750,0.891875,0.015000,-0.056250,-0.066875,-0.130000,0.116250,0.041875,5,7
2,2,Morocco,IR Iran,Group Phase,2018.0,0.0,1.0,41.0,686.0,37.0,...,-0.701250,-0.608750,0.289375,-0.327500,-0.082500,0.200000,0.236250,-0.388125,7,7
3,3,Portugal,Spain,Group Phase,2018.0,3.0,3.0,4.0,1274.0,10.0,...,0.513529,0.456471,-0.374706,-0.008824,-0.140000,-0.400000,0.145882,0.644706,8,5
4,4,France,Australia,Group Phase,2018.0,2.0,1.0,7.0,1198.0,36.0,...,0.031333,-0.022667,0.221333,-0.341333,0.240667,0.096000,-0.071333,0.004000,6,9


# Add the Continent

In [32]:
continent = continent.drop_duplicates()
df['Team1'] = df['Team1'].str.strip()
df['Team2'] = df['Team2'].str.strip()

continent['name'] = continent['name'].str.strip()
continent['Continent'] = continent['Continent'].str.strip()

In [33]:
df.loc[df.Year >= 1990,'continent_Team1'] = df['Team1'].map(continent.set_index('name')['Continent'])
df.loc[df.Year >= 1990,'continent_Team2'] = df['Team2'].map(continent.set_index('name')['Continent'])

In [34]:
df.head()

,Unnamed: 0,Team1,Team2,Rounds,Year,Score1,Score2,Ranking T1,Score T1,Ranking T2,...,clearances_z_y,blocks_z_y,aerials_z_y,fouls_z_y,fouled_z_y,nsxg_z_y,len_Team1,len_Team2,continent_Team1,continent_Team2
0,0,Russia,Saudi Arabia,Group Phase,2018.0,5.0,0.0,70.0,457.0,67.0,...,-0.246316,0.209474,-0.466316,-0.387368,0.051579,-0.294211,6,12,EU,AS
1,1,Egypt,Uruguay,Group Phase,2018.0,0.0,1.0,45.0,649.0,14.0,...,0.015000,-0.056250,-0.066875,-0.130000,0.116250,0.041875,5,7,AF,SA
2,2,Morocco,IR Iran,Group Phase,2018.0,0.0,1.0,41.0,686.0,37.0,...,0.289375,-0.327500,-0.082500,0.200000,0.236250,-0.388125,7,7,AF,AS
3,3,Portugal,Spain,Group Phase,2018.0,3.0,3.0,4.0,1274.0,10.0,...,-0.374706,-0.008824,-0.140000,-0.400000,0.145882,0.644706,8,5,EU,EU
4,4,France,Australia,Group Phase,2018.0,2.0,1.0,7.0,1198.0,36.0,...,0.221333,-0.341333,0.240667,0.096000,-0.071333,0.004000,6,9,EU,OC


In [35]:
# AF = Africa
# AS = Asia
# EU = Europe
# NA = North America
# SA = South America
# OC = Oceania
# AN = Antarctica

continent_dict = {"AF":1, "AS":2, "EU":3, "OC":4, "NA":5, "SA":6, "AN":7}
continent_encoded_team1 = []
continent_encoded_team2 = []


for i in range(len(df.continent_Team1)):
    continent_encoded_team1.append(continent_dict[list(df.continent_Team1)[i]])
    continent_encoded_team2.append(continent_dict[list(df.continent_Team2)[i]])
    
df["continent_en_team1"] = continent_encoded_team1
df["continent_en_team2"] = continent_encoded_team2

df = df.drop(['continent_Team1','continent_Team2'], axis =1)

In [36]:
df.head()

,Unnamed: 0,Team1,Team2,Rounds,Year,Score1,Score2,Ranking T1,Score T1,Ranking T2,...,clearances_z_y,blocks_z_y,aerials_z_y,fouls_z_y,fouled_z_y,nsxg_z_y,len_Team1,len_Team2,continent_en_team1,continent_en_team2
0,0,Russia,Saudi Arabia,Group Phase,2018.0,5.0,0.0,70.0,457.0,67.0,...,-0.246316,0.209474,-0.466316,-0.387368,0.051579,-0.294211,6,12,3,2
1,1,Egypt,Uruguay,Group Phase,2018.0,0.0,1.0,45.0,649.0,14.0,...,0.015000,-0.056250,-0.066875,-0.130000,0.116250,0.041875,5,7,1,6
2,2,Morocco,IR Iran,Group Phase,2018.0,0.0,1.0,41.0,686.0,37.0,...,0.289375,-0.327500,-0.082500,0.200000,0.236250,-0.388125,7,7,1,2
3,3,Portugal,Spain,Group Phase,2018.0,3.0,3.0,4.0,1274.0,10.0,...,-0.374706,-0.008824,-0.140000,-0.400000,0.145882,0.644706,8,5,3,3
4,4,France,Australia,Group Phase,2018.0,2.0,1.0,7.0,1198.0,36.0,...,0.221333,-0.341333,0.240667,0.096000,-0.071333,0.004000,6,9,3,4


In [37]:
df.isnull().sum()

Unnamed: 0            0
Team1                 0
Team2                 0
Rounds                0
Year                  0
Score1                0
Score2                0
Ranking T1            0
Score T1              0
Ranking T2            0
Score T2              0
goals_z_x             0
team_x                0
xg_z_x                0
crosses_z_x           0
boxtouches_z_x        0
passes_z_x            0
progpasses_z_x        0
takeons_z_x           0
progruns_z_x          0
tackles_z_x           0
interceptions_z_x     0
clearances_z_x        0
blocks_z_x            0
aerials_z_x           0
fouls_z_x             0
fouled_z_x            0
nsxg_z_x              0
goals_z_y             0
team_y                0
xg_z_y                0
crosses_z_y           0
boxtouches_z_y        0
passes_z_y            0
progpasses_z_y        0
takeons_z_y           0
progruns_z_y          0
tackles_z_y           0
interceptions_z_y     0
clearances_z_y        0
blocks_z_y            0
aerials_z_y     

# Add winner

In [38]:
#Adding who is the winner 
winner = []
Winner = []
for i in range (len(df['Team1'])):
    if df['Score1'][i] > df['Score2'][i]:
        winner.append(df['Team1'][i])
        Winner.append(1)
    elif df['Score1'][i] < df['Score2'][i]:
        winner.append(df['Team2'][i])
        Winner.append(2)
    else:
        winner.append('-1')
        Winner.append(0)

df['winner'] = winner
df['Winner'] = Winner
df.head()

,Unnamed: 0,Team1,Team2,Rounds,Year,Score1,Score2,Ranking T1,Score T1,Ranking T2,...,aerials_z_y,fouls_z_y,fouled_z_y,nsxg_z_y,len_Team1,len_Team2,continent_en_team1,continent_en_team2,winner,Winner
0,0,Russia,Saudi Arabia,Group Phase,2018.0,5.0,0.0,70.0,457.0,67.0,...,-0.466316,-0.387368,0.051579,-0.294211,6,12,3,2,Russia,1
1,1,Egypt,Uruguay,Group Phase,2018.0,0.0,1.0,45.0,649.0,14.0,...,-0.066875,-0.130000,0.116250,0.041875,5,7,1,6,Uruguay,2
2,2,Morocco,IR Iran,Group Phase,2018.0,0.0,1.0,41.0,686.0,37.0,...,-0.082500,0.200000,0.236250,-0.388125,7,7,1,2,IR Iran,2
3,3,Portugal,Spain,Group Phase,2018.0,3.0,3.0,4.0,1274.0,10.0,...,-0.140000,-0.400000,0.145882,0.644706,8,5,3,3,-1,0
4,4,France,Australia,Group Phase,2018.0,2.0,1.0,7.0,1198.0,36.0,...,0.240667,0.096000,-0.071333,0.004000,6,9,3,4,France,1


In [39]:
winner_counter = []
team1_winning=[]
team2_winning=[]

team1_winning_complete=[]
team2_winning_complete=[]


for i in range(len(df['Team1'])):
    if i != 0: 
        if df['Year'][i] != df['Year'][i-1]:
            for x in team1_winning:
                team1_winning_complete.append(x)
            for y in team2_winning:
                team2_winning_complete.append(y)


            winner_counter = []
            team1_winning=[]
            team2_winning=[]
            
    
        if ((df['winner'][i] != -1) & (df['winner'][i] == df['Team1'][i])):

            team2_winning.append(winner_counter.count(df['Team2'][i]))
            team1_winning.append(winner_counter.count(df['Team1'][i]))

            winner_counter.append(df['Team1'][i])

        elif((df['winner'][i] != -1) & (df['winner'][i] == df['Team2'][i])):

            team1_winning.append(winner_counter.count(df['Team1'][i]))
            team2_winning.append( winner_counter.count(df['Team2'][i]))


            winner_counter.append(df['Team2'][i])

        else:
            team1_winning.append(winner_counter.count(df['Team1'][i]))
            team2_winning.append(winner_counter.count(df['Team2'][i]))

    else:
        team1_winning.append(0)
        team2_winning.append(0)

        

for x in team1_winning:
    team1_winning_complete.append(x)
for y in team2_winning:
    team2_winning_complete.append(y)

   
    
            
df['team1_winning'] = team1_winning_complete
df['team2_winning'] = team2_winning_complete
        
        

In [40]:
df.shape

(888, 53)

In [41]:
df.head()

,Unnamed: 0,Team1,Team2,Rounds,Year,Score1,Score2,Ranking T1,Score T1,Ranking T2,...,fouled_z_y,nsxg_z_y,len_Team1,len_Team2,continent_en_team1,continent_en_team2,winner,Winner,team1_winning,team2_winning
0,0,Russia,Saudi Arabia,Group Phase,2018.0,5.0,0.0,70.0,457.0,67.0,...,0.051579,-0.294211,6,12,3,2,Russia,1,0,0
1,1,Egypt,Uruguay,Group Phase,2018.0,0.0,1.0,45.0,649.0,14.0,...,0.116250,0.041875,5,7,1,6,Uruguay,2,0,0
2,2,Morocco,IR Iran,Group Phase,2018.0,0.0,1.0,41.0,686.0,37.0,...,0.236250,-0.388125,7,7,1,2,IR Iran,2,0,0
3,3,Portugal,Spain,Group Phase,2018.0,3.0,3.0,4.0,1274.0,10.0,...,0.145882,0.644706,8,5,3,3,-1,0,0,0
4,4,France,Australia,Group Phase,2018.0,2.0,1.0,7.0,1198.0,36.0,...,-0.071333,0.004000,6,9,3,4,France,1,0,0


# Encode Country

In [42]:
countries = list(set(df["Team1"]).union(set(df["Team2"])))
countries_dict = {}
for i in range(len(countries)):
    countries_dict[countries[i]] = i
    
Team1_encoded = []
Team2_encoded = []

for i in range(len(df)):
    Team1_encoded.append(countries_dict[list(df.Team1)[i]])
    Team2_encoded.append(countries_dict[list(df.Team2)[i]])
    
df["Team1"] = Team1_encoded
df["Team2"] = Team2_encoded

In [43]:
df.head()

,Unnamed: 0,Team1,Team2,Rounds,Year,Score1,Score2,Ranking T1,Score T1,Ranking T2,...,fouled_z_y,nsxg_z_y,len_Team1,len_Team2,continent_en_team1,continent_en_team2,winner,Winner,team1_winning,team2_winning
0,0,48,23,Group Phase,2018.0,5.0,0.0,70.0,457.0,67.0,...,0.051579,-0.294211,6,12,3,2,Russia,1,0,0
1,1,2,45,Group Phase,2018.0,0.0,1.0,45.0,649.0,14.0,...,0.116250,0.041875,5,7,1,6,Uruguay,2,0,0
2,2,26,43,Group Phase,2018.0,0.0,1.0,41.0,686.0,37.0,...,0.236250,-0.388125,7,7,1,2,IR Iran,2,0,0
3,3,37,52,Group Phase,2018.0,3.0,3.0,4.0,1274.0,10.0,...,0.145882,0.644706,8,5,3,3,-1,0,0,0
4,4,59,0,Group Phase,2018.0,2.0,1.0,7.0,1198.0,36.0,...,-0.071333,0.004000,6,9,3,4,France,1,0,0


In [45]:
df = df.drop(['Score1','Score2','Rounds','winner'], axis= 1)

In [46]:
df.head()

,Unnamed: 0,Team1,Team2,Year,Ranking T1,Score T1,Ranking T2,Score T2,goals_z_x,team_x,...,fouls_z_y,fouled_z_y,nsxg_z_y,len_Team1,len_Team2,continent_en_team1,continent_en_team2,Winner,team1_winning,team2_winning
0,0,48,23,2018.0,70.0,457.0,67.0,465.0,0.315000,Russia,...,-0.387368,0.051579,-0.294211,6,12,3,2,1,0,0
1,1,2,45,2018.0,45.0,649.0,14.0,1018.0,-0.148750,Egypt,...,-0.130000,0.116250,0.041875,5,7,1,6,2,0,0
2,2,26,43,2018.0,41.0,686.0,37.0,708.0,-0.264375,Morocco,...,0.200000,0.236250,-0.388125,7,7,1,2,2,0,0
3,3,37,52,2018.0,4.0,1274.0,10.0,1126.0,0.110000,Portugal,...,-0.400000,0.145882,0.644706,8,5,3,3,0,0,0
4,4,59,0,2018.0,7.0,1198.0,36.0,718.0,0.037000,France,...,0.096000,-0.071333,0.004000,6,9,3,4,1,0,0


# Train & test

In [53]:
X_train = df.loc[(df['Year']!= 2018)]
X_test  = df.loc[(df['Year'] == 2018)]

y_train =  X_train["Winner"]
y_test  =  X_test["Winner"]


X_train = X_train.drop(['Winner','team_x','team_y'],axis=1)
X_test = X_test.drop(['Winner','team_x','team_y'],axis=1)

In [54]:
X_train.head()

,Unnamed: 0,Team1,Team2,Year,Ranking T1,Score T1,Ranking T2,Score T2,goals_z_x,xg_z_x,...,aerials_z_y,fouls_z_y,fouled_z_y,nsxg_z_y,len_Team1,len_Team2,continent_en_team1,continent_en_team2,team1_winning,team2_winning
64,64,16,25,2014.0,3.0,1242.0,18.0,903.0,0.063000,0.058000,...,0.311250,0.199375,-0.264375,0.115625,6,7,6,3,0,0
65,65,12,39,2014.0,20.0,882.0,56.0,558.0,0.082500,-0.142500,...,-0.411053,-0.235789,-0.197895,-0.176316,6,8,5,1,0,0
66,66,52,31,2014.0,1.0,1485.0,15.0,981.0,-0.031818,0.083636,...,0.030556,0.114444,-0.226111,-0.073333,5,11,3,3,0,0
67,67,18,0,2014.0,14.0,1026.0,62.0,526.0,0.177333,-0.029333,...,-0.130000,0.230000,-0.003529,-0.207647,5,9,6,4,0,0
68,68,62,42,2014.0,8.0,1137.0,12.0,1064.0,0.357500,0.026000,...,0.214118,-0.003529,0.428824,-0.044706,8,6,6,3,0,0


In [55]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()  
X_train = sc.fit_transform(X_train)  
X_test = sc.transform(X_test) 


import xgboost as xgb

dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

import xgboost as xgb
from sklearn.datasets import dump_svmlight_file

dump_svmlight_file(X_train, y_train, 'dtrain.svm', zero_based=True)
dump_svmlight_file(X_test, y_test, 'dtest.svm', zero_based=True)
dtrain_svm = xgb.DMatrix('dtrain.svm')
dtest_svm = xgb.DMatrix('dtest.svm')


[23:10:26] 760x46 matrix with 34960 entries loaded from dtrain.svm
[23:10:26] 128x46 matrix with 5888 entries loaded from dtest.svm


In [56]:
param = {
    'max_depth': 50,  # the maximum depth of each tree
    'eta': 0.005,  # the training step for each iteration
    'silent': 1,  # logging mode - quiet
    'objective': 'multi:softprob',  # error evaluation for multiclass training
    'num_class': 69}  # the number of classes that exist in this datset
num_round = 1000  # the number of training iterations

In [57]:
bst = xgb.train(param, dtrain, num_round)
preds = bst.predict(dtest)

import numpy as np
best_preds = np.asarray([np.argmax(line) for line in preds])


from sklearn.metrics import precision_score

print(accuracy_score(y_test, best_preds))

0.5390625


In [58]:
from sklearn import svm
model_svm = svm.SVC()
model_svm.fit(X_train,y_train)

y_pred_svm = model_svm.predict(X_test)

accuracy_score(y_test, y_pred_svm)

0.59375

In [59]:
model = Sequential()
model.add(Dense(12, input_dim=X_train.shape[1], activation='sigmoid'))
model.add(Dense(8, activation='sigmoid'))
#model.dropout()
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy']) #categorical crossentropy?

hist = model.fit(X_train, y_train, validation_data=(X_test,y_test), shuffle=True, epochs=100, batch_size=2)

Train on 760 samples, validate on 128 samples
Epoch 1/100
760/760 [==============================] - 2s 2ms/step - loss: 0.6159 - acc: 0.3224 - val_loss: 0.0424 - val_acc: 0.3984
Epoch 2/100
760/760 [==============================] - 1s 991us/step - loss: -0.1027 - acc: 0.3803 - val_loss: -0.3305 - val_acc: 0.3984
Epoch 3/100
760/760 [==============================] - 1s 1ms/step - loss: -0.3094 - acc: 0.3803 - val_loss: -0.5154 - val_acc: 0.3984
Epoch 4/100
760/760 [==============================] - 1s 1ms/step - loss: -0.4297 - acc: 0.3803 - val_loss: -0.6437 - val_acc: 0.3984
Epoch 5/100
760/760 [==============================] - 1s 1ms/step - loss: -0.5211 - acc: 0.3803 - val_loss: -0.7511 - val_acc: 0.3984
Epoch 6/100
760/760 [==============================] - 1s 1ms/step - loss: -0.6030 - acc: 0.3803 - val_loss: -0.8483 - val_acc: 0.3984
Epoch 7/100
760/760 [==============================] - 1s 1ms/step - loss: -0.6794 - acc: 0.3803 - val_loss: -0.9348 - val_acc: 0.3984
Epoch 8/1

Epoch 61/100
760/760 [==============================] - 1s 921us/step - loss: -3.6827 - acc: 0.3921 - val_loss: -2.7902 - val_acc: 0.3281
Epoch 62/100
760/760 [==============================] - 1s 925us/step - loss: -3.6928 - acc: 0.3908 - val_loss: -2.7502 - val_acc: 0.3203
Epoch 63/100
760/760 [==============================] - 1s 927us/step - loss: -3.7027 - acc: 0.3974 - val_loss: -2.7651 - val_acc: 0.3203
Epoch 64/100
760/760 [==============================] - 1s 989us/step - loss: -3.7100 - acc: 0.3947 - val_loss: -2.7665 - val_acc: 0.3203
Epoch 65/100
760/760 [==============================] - 1s 931us/step - loss: -3.7173 - acc: 0.3895 - val_loss: -2.7403 - val_acc: 0.3281
Epoch 66/100
760/760 [==============================] - 1s 928us/step - loss: -3.7324 - acc: 0.4053 - val_loss: -2.7626 - val_acc: 0.3203
Epoch 67/100
760/760 [==============================] - 1s 934us/step - loss: -3.7361 - acc: 0.3987 - val_loss: -2.7432 - val_acc: 0.3203
Epoch 68/100
760/760 [============

In [60]:
scores = model.evaluate(X_test, y_test)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

128/128 [==============================] - 0s 92us/step

acc: 31.25%
